# Transforming code into beautiful idiomatic python by Reymond Hettinger

### Notes from Reymond Hettinger Keynote from PyCon 2013

https://www.youtube.com/watch?v=OSGv2VnC0go

## General Notes

- Replace traditional index manipulation with Python's core looping idioms.
- Learn advanced techniques with for-else clauses and the two argument form of iter().
- One logical line of code equals one sentence in English.

## For Loops

In [ ]:
# Looping over a range of numbers

# Bad

for i in [0, 1, 2, 3, 4, 5]
    print(i)

# Good - Doesn't consume as much memory as a list, because it only stores start, stop and step values.

for i in range(6):
    print(i)


In [ ]:
# Looping backwards

colors = ['red', 'green', 'blue']

# Bad

for i in range(len(colors)-1, -1, -1):
    print(colors[i])

# Good

for color in reversed(colors):
    print(color)



In [ ]:
# Looping over a collection and indicies

colors = ['red', 'green', 'blue']

# Bad

for i in range(len(colors)):
    print(i, '-->', colors[i])

# Good

for i, color in enumerate(colors):
    print(i, '-->', color)



In [ ]:
# Looping over two collections

colors = ['red', 'green', 'blue']
names = ['piotr', 'julian', 'florian']

# Bad

n = min(len(names), len(colors))
for i in range(n):
    print(names[i], '-->', colors[n])

# Good

for name, color in zip(names, colors)
    print(name, '-->', color)



In [ ]:
# Looping in sorted order

colors = ['red', 'green', 'blue']

# Good

for color in sorted(colors):
    print(color)

# Reverse order

for color in sorted(colors, reverse=True):
    print(color)


In [ ]:
# Custom sort order

colors = ['red', 'green', 'blue']

# Bad

def compare_length(c1, c2):
    if len(c1) < len(c2): return -1
    if len(c1) > len(c2): return 1
    return 0

print(sorted(colors, cmp=compare_length))

# Good - Use Key Functions

print(sorted(colors, key=len))



In [ ]:
# Call a functions until a sentinel value

# Bad

blocks = []
while True:
    block = f.read(32)
    if block == '':
        break
    blocks.append(block)

# Good

blocks = []
for block in iter(partial(f.read, 32), ''):
    blocks.append(block)


In [ ]:
# Distinguish multiple exit points in loops

# Bad

def find(seq, target):
    found = False
    for i, value in enumerate(seq):
        if value == target:
            found = True
            break
    if not found:
        return -1
    return i

# Good

def find(seq, target):
    for i, value in enumerate(seq):
        if value == target:
            break
    else:
        return -1
    return i



## Dictionaries

In [ ]:
# Looping over dict keys

d = {'jeden': 1, 'dwa': 2, 'trzy': 3}

for k in d:
    print(k)

# If you're mutating the dict while looping on it you should do this instead:

for k in d.keys():
    if k.startswith('j'):
        del d[k]

In [ ]:
# Creating a dictionary from a list of keys.

print(dict.fromkeys(['piotr', 'julian', 'florian']))

In [ ]:
# Looping over dict keys and values

d = {'jeden': 1, 'dwa': 2, 'trzy': 3}

for k, v in d.items():
    print(k, '-->', v)



In [ ]:
# Construct a dictionary from pairs

colors = ['red', 'green', 'blue']
names = ['piotr', 'julian', 'florian']

d = dict(zip(colors, names))

# Using enumerate on a single collecion

d = dict(enumerate(names))


In [ ]:
# Counting with dictionaries

colors = ['red', 'green', 'blue']

# Good, but can be better

d = {}
for color in colors:
    if color not in d:
        d[color] = 0
    d[color] += 1

# Better

d = {}
for color in colors:
    d[color] = d.get(color, 0) + 1

# Beautiful - but get to know the defaultdict first.

from collections import defaultdict

d = defaultdict(int)
for color in colors:
    d[color] += 1



In [ ]:
# Grouping with dictionaries

colors = ['red', 'green', 'blue']

# Good, but can be better

d = {}
for color in colors:
    key = len(color)
    if key not in d:
        d[key] = []
    d[key].append(color)

# Better

d = {}
for color in colors:
    key = len(color)
    d.setdefault(key, []).append(color)

# Beautiful - but get to know the defaultdict first.

d = defaultdict(list)
for color in colors:
    key = len(color)
    d[key].append(color)



In [ ]:
# Is a dictionary popitem() atomic? - It is 

d = {'jeden': 1, 'dwa': 2, 'trzy': 3}

while d:
    key, value = d.popitem()
    print(key, '-->', value)

In [ ]:
# linking dictionaries together

defaults = {'jeden': 1, 'dwa': 2, 'trzy': 3}
parser = argparse.ArgumentParser()
parser.add_argument('-u', '--user')
parser.add_argument('-c', '--color')
namespace = parser.parse_args([])
command_line_args = {k:v for k, v in vars(namespace).items() if v}

# Old

d = defaults.copy()
d.update(os.environ)
d.update(command_line_args)

# Better

d = ChainMap(command_line_args, os.environ, defaults)



## General Stuff

In [ ]:
# Clarify function calls with keyword args.

# Bad

twitter_search('@obama', False, 20, True)

# Good

twitter_search('@obama', retweets=False, numtweets=20, popular=True)


In [ ]:
# Clarify multiple return values with namedtuples.

# Bad Example

doctest.testmod()
(0, 4)

# Good

doctest.testmod()
TestResults(failed=0, attempted=4)


In [ ]:
# Unpackig sequences

p = 'Reymond', 'Hettinger', 0x30, 'python@example.com'

# Bad

name = p[0]
surname = p[1]
age = p[2]
email = p[3]

# Good

name, surname, age, email = p

In [ ]:
# Updating multiple state variables

# Bad

def fibonacci(n):
    x = 0
    y = 1
    for i in range(n):
        print x
        t = y
        y = x + y
        x = t

# Good

def fibonacci(n):
    x, y = 0, 1
    for i in range(n):
        print x
        x, y = y, x+y

## Efficiency - Don't move data around unnecessarily.

In [ ]:
# Concatenating Strings

# Bad

x = 's' + 't'

# Good 

x = 's'.join('t')

In [ ]:
# Updating sequences

colors = ['red', 'green', 'blue']

# Bad

del colors[0]
colors.pop(0)
colors.insert(0, 'black')

# Good

from collections import deque

colors = deque(['red', 'green', 'blue'])

del colors[0]
colors.popleft(0)
colors.appendleft('black')
